In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [7]:
dat=pd.read_csv('./test/indian_food2.csv', # header=None,
                na_values=['nan', 'NA',' ','', '?'])
              #  encoding='utf8') # na_values=[' ','']
print(dat.columns)
print(dat.dtypes)

Index(['id', 'name', 'ingredients', 'diet', 'prep_time', 'cook_time',
       'flavor_profile', 'course', 'state', 'region'],
      dtype='object')
id                 int64
name              object
ingredients       object
diet              object
prep_time          int64
cook_time          int64
flavor_profile    object
course            object
state             object
region            object
dtype: object


# 전처리

In [4]:
pre1=dat.drop(columns='state')

In [5]:
pre2=pre1[(pre1.cook_time > 0)  &  (pre1.prep_time > 0)]

# pre2=pre1[pre1.cook_time > 0]
# pre2=pre2[pre2.prep_time > 0]

# pre2=pre1[(pre1[['cook_time', 'prep_time']] > 0).all(axis=1)]

In [8]:
pre2.region.unique()

array(['East', 'West', 'North', '-1', 'North East', 'South', 'Central',
       nan], dtype=object)

In [9]:
pre3=pre2[~pre2.region.isin(['-1', '', ' '])]
pre3['region']=pre3['region'].replace({'North East': 'North'})

pre3=pre3.dropna(subset=['region'])
len(pre3)

212

In [10]:
pre3.flavor_profile.unique()

array(['sweet', 'spicy', 'bitter', '-1', 'sour'], dtype=object)

In [12]:
pre4=pre3.copy()
pre4['ed_flavor_profile']=np.where(
          pre4.flavor_profile == 'sweet', 
            'sweet',
            'not sweet')
pre4['ed_course']=np.where(pre4.course == "main course", 
                           "main course",
                           "not main course")

In [13]:
pre4.ed_flavor_profile.unique()

array(['sweet', 'not sweet'], dtype=object)

In [14]:
pre4.ed_course.unique()

array(['not main course', 'main course'], dtype=object)

In [15]:
pre4.ingredients.head(5)

0                      Maida flour, yogurt, oil, sugar
1                              Gram flour, ghee, sugar
2         Carrots, milk, sugar, ghee, cashews, raisins
3    Flour, ghee, kewra, milk, clarified butter, su...
4    Milk powder, plain flour, baking powder, ghee,...
Name: ingredients, dtype: object

In [17]:
pre4.ingredients.str.split(',')

212

In [19]:
pre4['ingredients_no']=pre4.ingredients.apply(lambda x: len(x.split(',')))

In [20]:
basetable1=pre4.copy()

# Q1

In [21]:
tab=pd.crosstab(index=basetable1.ed_flavor_profile,
            columns=basetable1.region)

In [22]:
tab

region             Central  East  North  South  West
ed_flavor_profile                                   
not sweet                2     7     47     36    46
sweet                    1    20     14     19    20

In [23]:
from scipy.stats import chi2_contingency

In [24]:
q1=chi2_contingency(tab)
q1

(22.689047335851807,
 0.0001460865752415348,
 4,
 array([[ 1.95283019, 17.5754717 , 39.70754717, 35.80188679, 42.96226415],
        [ 1.04716981,  9.4245283 , 21.29245283, 19.19811321, 23.03773585]]))

In [25]:
np.trunc(q1[0]*1000)/1000

22.689

# Q2

In [26]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

In [ ]:
# from scipy.stats import chi2_contingency
# chi2_contingency()

# import scipy.stats as st
# st.chi2_contingency()

In [33]:
q2=ols('prep_time~region', data=basetable1).fit()
q2_out=anova_lm(q2)
q2_out

df        sum_sq       mean_sq         F    PR(>F)
region      4.0  5.409888e+04  13524.720512  2.254672  0.064429
Residual  207.0  1.241696e+06   5998.531150       NaN       NaN

In [34]:
q2_out['PR(>F)']

region      0.064429
Residual         NaN
Name: PR(>F), dtype: float64

In [37]:
import scipy.stats as st
basetable1.region.unique()

array(['East', 'West', 'North', 'South', 'Central'], dtype=object)

In [35]:
a1=basetable1[basetable1.region == 'East']['prep_time']
a2=basetable1[basetable1.region == 'West']['prep_time']
a3=basetable1[basetable1.region == 'North']['prep_time']
a4=basetable1[basetable1.region == 'South']['prep_time']
a5=basetable1[basetable1.region == 'Central']['prep_time']

q2_out2=st.f_oneway(a1, a2, a3, a4, a5)
q2_out2

F_onewayResult(statistic=2.254672047590146, pvalue=0.06442857691647358)

In [36]:
q2_out2.pvalue

0.06442857691647358

# Q3

In [38]:
pre7=pd.get_dummies(basetable1, 
                    columns=['ed_course', 'ed_flavor_profile'],
                    drop_first=True)

In [43]:
train=pre7[pre7['id'] % 3 != 0].reset_index(drop=True)
test=pre7[pre7['id'] % 3 == 0].reset_index(drop=True)

In [44]:
train.columns

Index(['id', 'name', 'ingredients', 'diet', 'prep_time', 'cook_time',
       'flavor_profile', 'course', 'region', 'ingredients_no',
       'ed_course_not main course', 'ed_flavor_profile_sweet'],
      dtype='object')

In [49]:
help(train.columns.str.replace)

Help on method replace in module pandas.core.strings:

replace(pat, repl, n=-1, case=None, flags=0, regex=True) method of pandas.core.strings.StringMethods instance
    Replace occurrences of pattern/regex in the Series/Index with
    some other string. Equivalent to :meth:`str.replace` or
    :func:`re.sub`.
    
    Parameters
    ----------
    pat : str or compiled regex
        String can be a character sequence or regular expression.
    
        .. versionadded:: 0.20.0
            `pat` also accepts a compiled regex.
    
    repl : str or callable
        Replacement string or a callable. The callable is passed the regex
        match object and must return a replacement string to be used.
        See :func:`re.sub`.
    
        .. versionadded:: 0.20.0
            `repl` also accepts a callable.
    
    n : int, default -1 (all)
        Number of replacements to make from start.
    case : bool, default None
        - If True, case sensitive (the default if `pat` is a strin

In [50]:
train.columns.str.replace(' ', '_')

Index(['id', 'name', 'ingredients', 'diet', 'prep_time', 'cook_time',
       'flavor_profile', 'course', 'region', 'ingredients_no',
       'ed_course_not_main_course', 'ed_flavor_profile_sweet'],
      dtype='object')

In [51]:
train.rename(columns={'ed_course_not main course': 'ed_course_not_main_course'})

id                name  \
0      1          Balu shahi   
1      2              Boondi   
2      4              Ghevar   
3      5         Gulab jamun   
4      7              Jalebi   
..   ...                 ...   
136  244  Mishti Chholar Dal   
137  248   Prawn malai curry   
138  250              Shukto   
139  251           Til Pitha   
140  254           Mawa Bati   

                                           ingredients            diet  \
0                      Maida flour, yogurt, oil, sugar      vegetarian   
1                              Gram flour, ghee, sugar      vegetarian   
2    Flour, ghee, kewra, milk, clarified butter, su...      vegetarian   
3    Milk powder, plain flour, baking powder, ghee,...      vegetarian   
4    Maida, corn flour, baking soda, vinegar, curd,...      vegetarian   
..                                                 ...             ...   
136  Chana dal, fresh coconut, ginger, cinnamon, ra...      vegetarian   
137      Coconut milk, prawns, garlic, turmeric, sugar  non vegetarian   
138  Green beans, bitter gourd, ridge gourd, banana...      vegetarian   
139            Glutinous rice, black sesame seeds, gur      vegetarian   
140  Milk powder, dry fruits, arrowroot powder, all...      vegetarian   

     prep_time  cook_time flavor_profile       course   region  \
0           45         25          sweet      dessert     East   
1           80         30          sweet      dessert     West   
2           15         30          sweet      dessert     West   
3           15         40          sweet      dessert     East   
4           10         50          sweet      dessert    North   
..         ...        ...            ...          ...      ...   
136         10         30          sweet  main course     East   
137         15         50          spicy  main course     East   
138         10         20          spicy  main course     East   
139          5         30          sweet      dessert    North   
140         20         45          sweet      dessert  Central   

     ingredients_no  ed_course_not_main_course  ed_flavor_profile_sweet  
0                 4                          1                        1  
1                 3                          1                        1  
2                10                          1                        1  
3                 8                          1                        1  
4                 9                          1                        1  
..              ...                        ...                      ...  
136               5                          0                        1  
137               5                          0                        0  
138               5                          0                        0  
139               3                          1                        1  
140               4                          1                        1  

[141 rows x 12 columns]

In [45]:
var_list=list(train.columns[-2:]) + ['prep_time']
var_list

['ed_course_not main course', 'ed_flavor_profile_sweet', 'prep_time']

In [47]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [48]:
lm=LinearRegression(fit_intercept=True)
lm.fit(train[var_list], train['cook_time'])

pred=lm.predict(test[var_list])

rmse=mean_squared_error(test['cook_time'], pred)**0.5

rmse

83.29102970312523